# Régression : Prédiction de force de morsure de lézards à partir de données anatomiques

#### **Auteurs** : Matéo PETITET, Mélodie FLEURY
#### **Matière** : UE Prog supervisée par Vincent GUIGUE - 3A IODAA AgroParisTech
#### **Date de rendue** : 18 novembre 2024

## Présentation des données

Les données utilisées sont issues de **Wittorsky & al., "Proximate determinants of bite force in Anolis lizards"** <span style="color: #BDBDBD">doi: 10.1111/joa.12394</span>. Il s'agit d'analyser des données morphologiques de lézards du genre *Anolis*, d'espèces et de sexe variables, afin de prédire la force de la morsure en Newtons. Ces données comportent des éléments morphologiques, musculaires, relatives à la masse, la longueur ou la surface. Il y a très peu d'exemples, 28, pour 60 attributs + la force de morsure. Les données sont organisées dans deux fichiers csv différents, l'un contenant les attributs utilisés pour la prédiction, et l'autre contenant une unique colonne de force de morsure.

L'enjeu est de réussir à faire de la sélection de variables et à travailler sur ce jeu de données avec un nombre d'exemples aussi faible. 

## Démarrage

Importons les bibliothèques qui nous serons utiles dans la suite, et chargeons les données avec un premier affichage.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#Chargement des données
X_lz = pd.read_csv('donnees_entrainement.csv')
y_lz = pd.read_csv('donnees_resultat.csv')

#Aperçu des données
apercu_attributs = X_lz.head()
info_attributs = X_lz.info()
apercu_cible = y_lz.head()
info_cible = y_lz.info()

print(apercu_attributs, info_attributs, apercu_cible, info_cible)

Effectuons un one-hot-encoding sur l'attribut de sexe, de sorte à pourvoir l'intégrer plus facilement aux analyses.

In [ ]:
#One-hot encoding sur 'Sex'
X_lz = pd.get_dummies(X_lz, columns=['Sex'], drop_first=True)

La colonne 'Sex' est remplacée par une colonne 'Sex_male' de valeur 0 ou 1.

## Nettoyage des valeurs

Procédons à présent au nettoyage des données, et particulièrement à la gestion des valeurs manquantes. Cette étape est essentielle pour mener toute analyse.

Affichons les valeurs manquantes :

In [ ]:
#Affichage du nombre de valeurs manquantes par colonne
missing_values = X_lz.isnull().sum()
print(missing_values[missing_values > 0])

En comparaison du nombre d'attributs, <span style="color: #159b36">il y a très peu de colonnes affectées par des valeurs manquantes </span>.

Au vu du faible nombre d'exemples, au-delà d'un quart de données manquantes, considérons que l'on n'utilisons pas la colonne ; en dessous, on va faire une imputation, d'autant plus pertinent que le seul cas restant une fois les colonnes lacunaires éliminées est une unique valeur manquante. 

In [ ]:
seuil=0.25*len(X_lz)

#Suppression des colonnes avec plus de 25% de valeurs manquantes
colonnes_a_supprimer = X_lz.columns[X_lz.isnull().sum() > seuil]
X_lz = X_lz.drop(columns=colonnes_a_supprimer)

print("Colonnes supprimées :", colonnes_a_supprimer)

Les colonnes trop vides ont été éliminées ; place à la gestion des valeurs manquantes restantes. Nous allons les remplaces par la moyenne des valeurs présentes sur la colonne.

In [ ]:
from sklearn.impute import SimpleImputer

#Imputation des valeurs manquantes des colonnes avec problèmes restantes
colonnes_a_imputer = X_lz.columns[X_lz.isnull().sum()>0]

imputer = SimpleImputer(strategy='mean')    #on utilise la moyenne

if len(colonnes_a_imputer)>0:
    for colonne in colonnes_a_imputer:
        X_imput=imputer.fit_transform(X_lz[[colonne]])
        X_lz[[colonne]] = pd.DataFrame(X_imput, columns=[colonne], index=X_lz.index)


#Vérifions qu'il n'y a plus de valeurs manquantes
print(X_lz.isnull().sum())

Il n'y a à présent <span style="color: #159b36">plus aucune valeur manquante, nous pouvons passer à l'analyse des données</span>. Il est à noter qu'au vu de la taille du jeu de données, tout ce nettoyage aurait pu facilement être réalisé à la main. Nous avons cependant ici un bout de script qui pourra être réutilisé dans d'autres contextes plus complexes.

## Recherche de corrélations

Une première étape pour la sélection de variables est la recherche de corrélations entre les attributs et la force de morsure.

In [ ]:
corr = np.abs(X_lz.select_dtypes(include=[np.float64]).T@y_lz)
noms=list(X_lz.columns)
noms.pop(0)     #on retire la colonne de noms d'espèces
noms.pop(-1)    #on retire la colonne du sexe
plt.figure(figsize=(12, 6))
plt.xticks(rotation=45, ha='right')
plt.bar(noms, corr['bite force_w'].to_numpy())

Conclusions :

- <span style="color: #159b36">assez logiquement, les données de dissections et les données de terrain donnent la même forme ; on retirera les données de terrain</span>

- <span style="color: #159b36"> quelques variables semblent se dégager, mais probablement plus à cause de leur valeur absolue par rapport aux autres</span>

Retirons donc les données de terrain, ainsi que le nom des espèces qui joue plus un rôle d'identifiant dans ce contexte.

In [ ]:
X_lz=X_lz.drop(columns= ['SVL_w', 'headl_w', 'headW_w', 'headh_w', 'lower jawl_w', 'tip-quadr_w', 'tip-coron_w', 'opening_w', 'closing_w'])
X_lz=X_lz.drop(columns= ['Species'])

Maintenant que les colonnes superflues ont été retirées, observons à nouveau les corrélations avec une matrice de corrélation, plus lisible.

In [ ]:
import seaborn as sns

data=X_lz.copy()    #dopy très important pour ne pas modifier X_lz !
data['bite_force_w'] = y_lz['bite force_w']

corr_2=data.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr_2, cmap='coolwarm', linewidths=0.5)   #pour une matrice plus agréable à lire
plt.title("Corrélation des caractéristiques", fontsize=16)
plt.xticks(rotation=45, ha='right') 
plt.show()

<span style="color: #159b36">De nouvelles variables se dégagent, et notamment celles relative à la masse des muscles ainsi que leur section. Ces observations sont en accord avec les conclusions de l'article dont sont issues les données.</span>

## Sélection de variables avec le SequentialFeatureSelector

Regardons si le SFS peut extraire un certain nombre de variables. L'objectif est d'en avoir 7 au final, et d'ajouter dedans le sexe ; nous en cherchons donc 6. Cela permet d'avoir un nombre de variables plus faible que le nombre d'exemple, 4 fois plus faible précisément.

Après avoir réalisé beaucoup d'essais en changeant de modèles, de plis pour la validation croisée et de direction, choisissons d'utiliser un <span style="color: #159b36">Random Forest avec 5 plis en direction "forward"</span>.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SequentialFeatureSelector

estimator = RandomForestRegressor()
selector = SequentialFeatureSelector(estimator, n_features_to_select=6, cv=5)
selector = selector.fit(X_lz, y_lz)
selected_features = X_lz.columns[selector.get_support(indices=True)]

In [ ]:
print(selected_features)

<span style="color: #159b36">À chaque essai, les features sélectionnées changent, il n'y a aucune stabilité dans la sortie. Aussi, pour la sélection finale et faciliter la reproduction des résultats, fixons les variables ayant données les meilleurs résultats en apprentissage, qui sont celles se dégageant le plus dans la matrice de corrélation.</span>

In [ ]:
selected_features=['headh', 'DM_muscle_mass', 'AMESP_muscle_mass', 'PSTP_muscle_mass', 'PSTP_physiological_cross_sectional_area', 'AMP_physiological_cross_sectional_area']     #choix basé sur ce qui revenait régulièrement lors de mes essais + matrice de corrélation

Xf=[]
X_final=pd.DataFrame(Xf)
for attribut in selected_features:
    X_final[attribut]=X_lz[attribut]
X_final['Sex_male']=X_lz['Sex_male'] #choix d'ajouter sexe car j'ai constaté que c'est important

## Début de l'apprentissage

Recherchons le modèle donnant les meilleurs résultats bruts parmis un ensemble de modèles possibles. Les modèles testés sont la régression linéaire, ridge, lasso, ElasticNet, les k plus proches voisins, SVR, random forest et XGBoost. On effectue un leave-one-out au vu du très faible nombre d'exemples à notre disposition. La métrique d'évaluation est l'erreur quadratique moyenne (en anglais Mean Squared Error, MSE), soit la moyenne des carrés des erreurs entre les prédictions et les valeurs réelles. L'implémentation du MSE dans sckiit-learn est telle qu'elle est donnée négativement, il faut bien prendre cela en compte.

In [ ]:
scores_moyens=[]   #stockons les résultats

from sklearn.model_selection import LeaveOneOut, cross_val_score
loo = LeaveOneOut()

from sklearn.linear_model import LinearRegression
model = LinearRegression()
scores = cross_val_score(model, X_final, y_lz, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens.append('LR')
scores_moyens.append(mean_score)

from sklearn.linear_model import Ridge
ridge_model = Ridge()
scores = cross_val_score(ridge_model, X_final, y_lz, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens.append('Ridge')
scores_moyens.append(mean_score)

from sklearn.linear_model import Lasso
lasso_model = Lasso()
scores = cross_val_score(lasso_model, X_final, y_lz, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens.append('Lasso')
scores_moyens.append(mean_score)

from sklearn.linear_model import ElasticNet
elastic_model = ElasticNet()
scores = cross_val_score(elastic_model, X_final, y_lz, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens.append('ElasticNet')
scores_moyens.append(mean_score)

from sklearn.neighbors import KNeighborsRegressor
knn_model = KNeighborsRegressor()
scores = cross_val_score(knn_model, X_final, y_lz, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens.append('KNN')
scores_moyens.append(mean_score)

from sklearn.svm import SVR
svr_model = SVR(kernel='linear') #arbitrairement, linéaire
scores = cross_val_score(svr_model, X_final, y_lz, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens.append('SVR')
scores_moyens.append(mean_score)

rfr_model = RandomForestRegressor()
scores = cross_val_score(rfr_model, X_final, y_lz, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens.append('RFR')
scores_moyens.append(mean_score)

from xgboost import XGBRegressor
xgb_model = XGBRegressor()
scores = cross_val_score(xgb_model, X_final, y_lz, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens.append('XGB')
scores_moyens.append(mean_score)

In [ ]:
print(scores_moyens)

Nous constatons que <span style="color: #159b36">le SVR est, et de loin, le meilleur modèle. Ce modèle est optimisable ; cherchons donc à l'améliorer à l'aide d'optuna, et optimisons également les trois modèles suivant en performance pour voir si l'un deux prendra le dessus une fois optimisé</span>.
## Optimisation de modèles


Nous allons chercher à optimiser chaque paramètre du SVR, ridge, ElasticNet et lasso.

In [ ]:
import optuna

#Optimisation SVR
def objective_svr(trial):
# Hyperparamètres à optimiser
    params = {
        'C': trial.suggest_loguniform('C', 1e-3, 1e3),
        'epsilon': trial.suggest_loguniform('epsilon', 1e-4, 1e1),
        'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
    }

#Paramètres supplémentaires pour les kernels 'poly' et 'rbf'
    if params['kernel'] == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 5)
        params['coef0'] = trial.suggest_uniform('coef0', 0.0, 10.0)
    if params['kernel'] in ['poly', 'rbf', 'sigmoid']:
        params['gamma'] = trial.suggest_loguniform('gamma', 1e-4, 1e1)

#Création et entraînement du modèle
    model = SVR(**params)
    scores = cross_val_score(model, X_final, y_lz, cv=loo, scoring='neg_mean_squared_error')

# Retourner la moyenne de l'erreur quadratique moyenne négative
    return np.mean(scores)

study_svr = optuna.create_study(direction='maximize')   #maximisation de la métrique d'évaluation car elle est donnée négative
study_svr.optimize(objective_svr, n_trials=250)

In [ ]:
#Optimisation ridge
def objective_ridge(trial):
# Hyperparamètre à optimiser
    alpha = trial.suggest_loguniform('alpha', 1e-4, 1e3)

#Création et entraînement du modèle
    model = Ridge(alpha=alpha)
    scores = cross_val_score(model, X_final, y_lz, cv=loo, scoring='neg_mean_squared_error')

# Retourner la moyenne de l'erreur quadratique moyenne négative
    return np.mean(scores)

study_ridge = optuna.create_study(direction='maximize')   #maximisation de la métrique d'évaluation car elle est donnée négative
study_ridge.optimize(objective_ridge, n_trials=250)

In [ ]:
def objective_elasticnet(trial):
# Hyperparamètres à optimiser
    params = {
        'alpha': trial.suggest_loguniform('alpha', 1e-4, 1e2),
        'l1_ratio': trial.suggest_uniform('l1_ratio', 0.0, 1.0)
    }

#Création et entraînement du modèle
    model = ElasticNet(**params, max_iter=10000)
    scores = cross_val_score(model, X_final, y_lz, cv=loo, scoring='neg_mean_squared_error')

# Retourner la moyenne de l'erreur quadratique moyenne négative
    return np.mean(scores)

study_elasticnet = optuna.create_study(direction='maximize')   #maximisation de la métrique d'évaluation car elle est donnée négative
study_elasticnet.optimize(objective_elasticnet, n_trials=250)

In [ ]:
def objective_lasso(trial):
# Hyperparamètre à optimiser
    alpha = trial.suggest_loguniform('alpha', 1e-4, 1e2)

#Création et entraînement du modèle Lasso
    model = Lasso(alpha=alpha, max_iter=10000)
    scores = cross_val_score(model, X_final, y_lz, cv=loo, scoring='neg_mean_squared_error')

# Retourner la moyenne de l'erreur quadratique moyenne négative
    return np.mean(scores)

study_lasso = optuna.create_study(direction='maximize')   #maximisation de la métrique d'évaluation car elle est donnée négative
study_lasso.optimize(objective_lasso, n_trials=250)

In [ ]:
print("Meilleurs hyperparamètres pour SVR :", study_svr.best_params)
print("Meilleure valeur de MSE pour SVR :", study_svr.best_value)
print("Meilleurs hyperparamètres pour ridge :", study_ridge.best_params)
print("Meilleure valeur de MSE pour ridge :", study_ridge.best_value)
print("Meilleurs hyperparamètres pour elasticnet :", study_elasticnet.best_params)
print("Meilleure valeur de MSE pour elasticnet :", study_elasticnet.best_value)
print("Meilleurs hyperparamètres pour lasso :", study_lasso.best_params)
print("Meilleure valeur de MSE pour lasso :", study_lasso.best_value)

Après optimisation, <span style="color: #159b36">le SVR reste le meilleur des modèles. Nous sommes en mesure de faire passer le MSE de environ 17 à environ 12,5 avec les paramètres suivants : un noyau poly, C = 1.216712493672062, epsilon = 0.5854463926783681, degree = 2, coef0 = 8.613116087425539 et gamma = 0.017014770725644102. Cela donne un écart moyen entre la valeur réelle et la valeur prédite d'environ 3,5 newtons. Une grande partie des valeurs de force de morsure se situe en-dessous de cette erreur, et une autre grande partie est proportionnellement petite par rapport à l'erreur. La prédiction est donc peu fiable dans la plupart des cas. Cela peut s'expliquer par la présence de quelques exemples avec une force remarquablement plus importante que les autres ; leur écart doit fausser l'apprentissage. Notre modèle gagnerait probablement à être entraîné sur des jeux de données de lézards forts et faibles, et serait probablement très bon dans ce contexte. Dans le cadre d'un jeu de données avec de grands écarts, les performances sont donc globalement très correctes. </span>
## Subdivision du jeu de données

Retirons les colonnes avec la plus grande valeur de force pour voir si retirer les outliers rends la prédiction meilleure.

In [ ]:
valeurs_morsures_triees = y_lz['bite force_w'].sort_values(ascending=False)  # Convertir en liste si besoin
print(valeurs_morsures_triees)

In [ ]:
index = y_lz.index
valeurs = y_lz['bite force_w']

plt.figure(figsize=(10, 6))
plt.scatter(index, valeurs, color='blue', alpha=0.7)
plt.xlabel("Index")
plt.ylabel("Valeur de force de morsure")
plt.grid(True)
plt.show()

Une valeurs se dégage particulièrement, à laquelle on peut en ajouter cinq autres au-dessus de la masse ; retirons-les. Au vu de leur faible nombre, il ne serait pas pertinent de les mettre dans un nouveau jeu de données à explorer séparément.

In [ ]:
index_a_supprimer = y_lz['bite force_w'].nlargest(6).index
print(index_a_supprimer)
X_faibles=X_final.copy()
y_faibles=y_lz.copy()
X_faibles=X_faibles.drop(index=index_a_supprimer)
y_faibles=y_faibles.drop(index=index_a_supprimer)

Nous avons nos nouvelles données avec les outliers retirés. Les conclusions précédentes sur le SVR et les optimisations ayant été obtenues à partir des données intégrales, recommençons le processus précédent de sélection de modèles.

In [ ]:
scores_moyens_2=[]   #stockons les résultats

#LR
model = LinearRegression()
scores = cross_val_score(model, X_faibles, y_faibles, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens_2.append('LR')
scores_moyens_2.append(mean_score)

#Ridge
ridge_model = Ridge()
scores = cross_val_score(ridge_model, X_faibles, y_faibles, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens_2.append('Ridge')
scores_moyens_2.append(mean_score)

#Lasso
lasso_model = Lasso()
scores = cross_val_score(lasso_model, X_faibles, y_faibles, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens_2.append('Lasso')
scores_moyens_2.append(mean_score)

#ElasticNet
elastic_model = ElasticNet()
scores = cross_val_score(elastic_model, X_faibles, y_faibles, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens_2.append('ElasticNet')
scores_moyens_2.append(mean_score)

#KNN
knn_model = KNeighborsRegressor()
scores = cross_val_score(knn_model, X_faibles, y_faibles, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens_2.append('KNN')
scores_moyens_2.append(mean_score)

#SVR
svr_model = SVR(kernel='linear') #arbitrairement, linéaire
scores = cross_val_score(svr_model, X_faibles, y_faibles, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens_2.append('SVR')
scores_moyens_2.append(mean_score)

#RFR
rfr_model = RandomForestRegressor()
scores = cross_val_score(rfr_model, X_faibles, y_faibles, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens_2.append('RFR')
scores_moyens_2.append(mean_score)

#XGB
xgb_model = XGBRegressor()
scores = cross_val_score(xgb_model, X_faibles, y_faibles, cv=loo, scoring='neg_mean_squared_error')
mean_score = np.mean(scores)
scores_moyens_2.append('XGB')
scores_moyens_2.append(mean_score)

In [ ]:
print(scores_moyens_2)

<span style="color: #159b36">Les performances sont d'entrée de jeu bien meilleures et bien plus serrées, avec cette fois-ci ElasticNet qui se dégage, suivi de près par lasso et d'un peu plus loin par random forest. </span> Essayons d'optimiser ces trois modèles avec optuna.

In [ ]:
def objective_elasticnet_2(trial):
# Hyperparamètres à optimiser
    params = {
        'alpha': trial.suggest_loguniform('alpha', 1e-4, 1e2),
        'l1_ratio': trial.suggest_uniform('l1_ratio', 0.0, 1.0)
    }

#Création et entraînement du modèle
    model = ElasticNet(**params, max_iter=10000)
    scores = cross_val_score(model, X_faibles, y_faibles, cv=loo, scoring='neg_mean_squared_error')

# Retourner la moyenne de l'erreur quadratique moyenne négative
    return np.mean(scores)

study_elasticnet_2 = optuna.create_study(direction='maximize')   #maximisation de la métrique d'évaluation car elle est donnée négative
study_elasticnet_2.optimize(objective_elasticnet_2, n_trials=250)

In [ ]:
def objective_lasso_2(trial):
# Hyperparamètre à optimiser
    alpha = trial.suggest_loguniform('alpha', 1e-4, 1e2)

#Création et entraînement du modèle Lasso
    model = Lasso(alpha=alpha, max_iter=10000)
    scores = cross_val_score(model, X_faibles, y_faibles, cv=loo, scoring='neg_mean_squared_error')

# Retourner la moyenne de l'erreur quadratique moyenne négative
    return np.mean(scores)

study_lasso_2 = optuna.create_study(direction='maximize')   #maximisation de la métrique d'évaluation car elle est donnée négative
study_lasso_2.optimize(objective_lasso_2, n_trials=250)

In [ ]:
def objective_rfr(trial):
# Hyperparamètres à optimiser
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

#Création et entraînement du modèle
    model = RandomForestRegressor(**params, random_state=42)
    scores = cross_val_score(model, X_faibles, y_faibles, cv=loo, scoring='neg_mean_squared_error')

# Retourner la moyenne de l'erreur quadratique moyenne
    return np.mean(scores)

study_rfr = optuna.create_study(direction='maximize')   #maximisation de la métrique d'évaluation car elle est donnée négative
study_rfr.optimize(objective_rfr, n_trials=250)

In [ ]:
print("Meilleurs hyperparamètres pour ElasticNet :", study_elasticnet_2.best_params)
print("Meilleure valeur de MSE pour ElasticNet :", study_elasticnet_2.best_value)
print("Meilleurs hyperparamètres pour lasso :", study_lasso_2.best_params)
print("Meilleure valeur de MSE pour lasso :", study_lasso_2.best_value)
print("Meilleurs hyperparamètres pour random forest :", study_rfr.best_params)
print("Meilleure valeur de MSE pour random forest :", study_rfr.best_value)

<span style="color: #159b36">ElasticNet reste le meilleur modèle après optimisation ; nous sommes en mesure de lui faire gagner environ 0,2 points avec les paramètres suivants : alpha = 1.913255964637761 et l1_ratio = 2.7280103109729708e-05. Cette amélioration est satisfaisante au vue de la faible valeur initiale de MSE. Notre modèle final devient donc bien meilleur une fois les données séparées des outliers. Considérant que l'erreur est la moyenne des écarts au carré, les performances sont finalement très bonnes. </span>
## Conclusion

<span style="color: #159b36">Considérant la taille très réduite du jeu de données et le nombre d'attribut, le modèle final parvient à estimer la force de morsure de lézards de manière assez satisfaisante. Nous avons vu l'importance de l'homogénéité des données dans ce contexte, au point de changer de modèle de prédilection une fois celles-ci séparées des outliers.</span>